In [1]:
import pandas as pd
import numpy as np

In [5]:
recipes = pd.read_parquet('../data_sources/recipepairs/recipes.parquet') 
pairs = pd.read_parquet('../data_sources/recipepairs/pairs.parquet') 

In [6]:
recipes.head()

,id,name,steps,ingredients,categories,name_tokens
0,96313,grilled garlic cheese grits,"i a sauce pan, bring water to a boil; slowly a...","[cheddar, garlic, grit, olive oil, salt, water]","[low_calorie, low_sugar, gluten_free, low_carb...","[grill, garlic, cheese, grit]"
1,232037,simple shrimp and andouille jambalaya,"in a food processor, pulse the onion, red pepp...","[bay leafe, garlic, hot sauce, mollusc, onion,...","[dairy_free, gluten_free]","[simple, shrimp, and, andouille, jambalaya]"
2,232047,beef stew with dried cherries,preheat oven to 350°f. cut beef into 1 inch cu...,"[allspice, beef stew meat, beef stock, cherry,...","[dairy_free, gluten_free]","[beef, stew]"
3,232050,hot sweet almond brittle,preheat oven to 375°f place almonds in single ...,"[almond, coriander, cumin, pepper, salt, sugar...","[vegetarian, dairy_free, gluten_free, vegan]","[hot, sweet, almond, brittle]"
4,232076,retro chicken and chips casserole,"in large bowl, combine chicken, green onion, b...","[cheddar, chicken, dijon mustard, lemon juice,...",[gluten_free],"[retro, chicken, and, chip, casserole]"


In [8]:
recipes.shape

(459052, 6)

In [9]:
recipes.columns

Index(['id', 'name', 'steps', 'ingredients', 'categories', 'name_tokens'], dtype='object')

In [13]:
recipes.isna().sum()

id             0
name           0
steps          0
ingredients    0
categories     0
name_tokens    0
dtype: int64

In [ ]:
# unique categories
np.unique(np.concatenate(recipes['categories']))

array(['dairy_free', 'gluten_free', 'low_calorie', 'low_carb', 'low_fat',
       'low_sodium', 'low_sugar', 'vegan', 'vegetarian'], dtype=object)

In [7]:
pairs.head()

,base,target,name_overlap,name_iou,categories
0,96313,86230,grilled garlic ch,0.5312,['dairy_free']
1,86230,96313,grilled garlic ch,0.5312,"['low_calorie', 'low_sugar', 'vegetarian']"
2,107592,96313,garlic cheese grits,0.7037,"['low_calorie', 'low_sugar']"
3,442284,96313,garlic cheese grits,0.5135,"['low_calorie', 'low_carb', 'low_sugar', 'vege..."
4,96313,379443,grilled garlic ch,0.5312,['dairy_free']


In [10]:
pairs.shape

(6908697, 5)

In [12]:
pairs.columns

Index(['base', 'target', 'name_overlap', 'name_iou', 'categories'], dtype='object')

In [14]:
pairs.isna().sum()

base            0
target          0
name_overlap    0
name_iou        0
categories      0
dtype: int64

In [22]:
from collections import Counter

Counter(pairs['categories'])

Counter({"['gluten_free']": 996935,
         "['dairy_free']": 968348,
         "['low_sodium']": 528340,
         "['vegetarian']": 510143,
         "['low_carb']": 473930,
         "['dairy_free', 'vegan']": 294209,
         "['low_calorie', 'low_carb']": 185653,
         "['low_calorie']": 158868,
         "['dairy_free', 'gluten_free']": 155282,
         "['low_fat']": 147363,
         "['low_calorie', 'low_carb', 'low_sodium']": 131986,
         "['low_sugar']": 116917,
         "['gluten_free', 'low_carb']": 109880,
         "['vegan', 'vegetarian']": 99173,
         "['gluten_free', 'low_sodium']": 98908,
         "['low_carb', 'low_sodium']": 86869,
         "['low_calorie', 'low_fat']": 84234,
         "['low_calorie', 'low_fat', 'low_sodium']": 73794,
         "['dairy_free', 'vegan', 'vegetarian']": 73584,
         "['low_calorie', 'low_sodium']": 70228,
         "['vegan']": 67652,
         "['gluten_free', 'vegetarian']": 65433,
         "['dairy_free', 'low_sodium']": 516

In [ ]:
pairs.loc[0]

base                        96313
target                      86230
name_overlap    grilled garlic ch
name_iou                   0.5312
categories         ['dairy_free']
Name: 0, dtype: object

In [35]:
recipes.loc[recipes['id'] == pairs.loc[0].base]

,id,name,steps,ingredients,categories,name_tokens
0,96313,grilled garlic cheese grits,"i a sauce pan, bring water to a boil; slowly a...","[cheddar, garlic, grit, olive oil, salt, water]","[low_calorie, low_sugar, gluten_free, low_carb...","[grill, garlic, cheese, grit]"


In [36]:
recipes.loc[recipes['id'] == pairs.loc[0].target]

,id,name,steps,ingredients,categories,name_tokens
6723,86230,grilled garlic chicken,"in medium bowl, combine soup mix with oil. add...","[chicken, mushroom, oil]","[dairy_free, gluten_free, low_carb]","[grill, garlic, chicken]"


Several of the name overlaps here are for recipes that are clearly distinct. Lets play around with name overlap to try and get better pairs

In [37]:
pairs_subset = pairs[pairs['name_iou'] > 0.7]
pairs_subset.head()

,base,target,name_overlap,name_iou,categories
2,107592,96313,garlic cheese grits,0.7037,"['low_calorie', 'low_sugar']"
9,67385,96313,garlic cheese grits,0.7037,"['low_calorie', 'low_carb', 'low_sugar', 'vege..."
10,155316,96313,garlic cheese grits,0.7037,"['low_calorie', 'low_carb', 'low_sugar']"
14,202932,96313,garlic cheese grits,0.7037,"['low_calorie', 'low_carb', 'low_sugar']"
15,472759,96313,ed garlic cheese grits,0.7333,"['low_calorie', 'low_sugar', 'vegetarian']"


In [38]:
pairs_subset.shape

(939922, 5)

In [39]:
Counter(pairs_subset['categories'])

Counter({"['dairy_free']": 132382,
         "['gluten_free']": 132372,
         "['low_carb']": 81168,
         "['low_sodium']": 80889,
         "['vegetarian']": 63374,
         "['low_calorie', 'low_carb']": 34039,
         "['dairy_free', 'vegan']": 33742,
         "['low_calorie']": 26332,
         "['low_calorie', 'low_carb', 'low_sodium']": 21840,
         "['low_fat']": 21784,
         "['low_sugar']": 16870,
         "['gluten_free', 'low_carb']": 16774,
         "['dairy_free', 'gluten_free']": 16045,
         "['low_calorie', 'low_fat']": 13058,
         "['low_carb', 'low_sodium']": 12973,
         "['vegan', 'vegetarian']": 11842,
         "['gluten_free', 'low_sodium']": 11664,
         "['low_calorie', 'low_fat', 'low_sodium']": 10615,
         "['low_calorie', 'low_sodium']": 10420,
         "['low_calorie', 'low_carb', 'low_fat']": 8563,
         "['vegan']": 8470,
         "['dairy_free', 'vegan', 'vegetarian']": 7946,
         "['low_calorie', 'low_carb', 'low_fat', 

In [46]:
pairs_subset.iloc[552]

base                           206237
target                         260554
name_overlap    chicken tortilla soup
name_iou                       0.7778
categories             ['dairy_free']
Name: 3865, dtype: object

In [47]:
recipes.loc[recipes['id'] == pairs_subset.iloc[552].base]

,id,name,steps,ingredients,categories,name_tokens
175100,206237,texas chicken tortilla soup,"in a large saucepan, combine the first ten ing...","[bean, carrot, cheese, chicken, chicken broth,...",[],"[texas, chicken, tortilla, soup]"


In [48]:
recipes.loc[recipes['id'] == pairs_subset.iloc[552].target]

,id,name,steps,ingredients,categories,name_tokens
153,260554,chicken tortilla soup,heat the oil in a medium saucepan over medium ...,"[chicken, chili powder, chipotle chile, corian...",[dairy_free],"[chicken, tortilla, soup]"


In [61]:
recipes.loc[recipes['id'] == pairs_subset.iloc[552].target]['ingredients'].explode().tolist()

['chicken',
 'chili powder',
 'chipotle chile',
 'coriander',
 'corn kernel',
 'garlic',
 'lime juice',
 'lime wedge',
 'low sodium chicken broth',
 'onion',
 'salt',
 'tomato',
 'tortilla',
 'vegetable oil']

Try to automatically extract recipes by restrictions

In [62]:
pairs_vegan = pairs_subset[pairs_subset['categories'].apply(lambda x: 'vegan' in x)]
pairs_vegan.head()

,base,target,name_overlap,name_iou,categories
19,341532,232050,sweet almond brittle,0.8000,"['dairy_free', 'vegan']"
808,248232,387267,chocolate biscuit,0.7083,"['dairy_free', 'low_sugar', 'vegan']"
1241,342209,213878,banana oatmeal cookies,1.0000,"['dairy_free', 'low_sodium', 'vegan']"
1257,429673,387908,chinese salad,1.0000,"['dairy_free', 'vegan']"
1320,489197,456296,pasta primavera,1.0000,"['dairy_free', 'vegan']"


In [63]:
pairs_vegan.shape

(98121, 5)

In [64]:
pairs_vegetarian = pairs_subset[pairs_subset['categories'].apply(lambda x: 'vegetarian' in x)]
pairs_vegetarian.shape

(129677, 5)

In [65]:
pairs_df = pairs_subset[pairs_subset['categories'].apply(lambda x: 'dairy_free' in x)]
pairs_df.shape

(266503, 5)

Testing out compression

In [68]:
pairs_vegan_compressed = pairs_vegan.groupby('base', as_index=False).agg({'target': list})
pairs_vegan_compressed.head()

,base,target
0,54,"[154898, 225429, 228477, 419515, 530682]"
1,60,"[21117, 68368]"
2,68,[130896]
3,70,[278946]
4,80,[14237]


In [69]:
pairs_vegan_compressed.shape

(30393, 2)

In [73]:
def get_item_by_id(id):
	return recipes.loc[recipes['id'] == id]

In [74]:
get_item_by_id(232088)

,id,name,steps,ingredients,categories,name_tokens
9,232088,potato chip chocolate chip cookies,position racks in the top and bottom thirds of...,"[baking soda, brown sugar, chocolate chip, egg...","[vegetarian, dairy_free, gluten_free]","[potato, chip, chocolate, chip, cookie]"


In [75]:
get_item_by_id(523902)

,id,name,steps,ingredients,categories,name_tokens
202079,523902,vegan chocolate chip cookies,preheat oven to 375. create chia eggs by combi...,"[applesauce, baking soda, chia seed, chocolate...","[vegetarian, dairy_free, gluten_free, vegan]","[vegan, chocolate, chip, cookie]"
